## Stammzelltisch 21.5.2024 - Large language model tutorial

## Background
* There are a couple of different large language models available
    * OpenAI models
        * GPT-3.5
        * GPT-4.0
    * LLama2
        * 7, 13, 70 billion parameters
        * Here 7 billion parameters, 4-bits --> can be run on laptop with 8 GBs
    * Gemma
    * ...

### Initialize large language model
* Important: Ollama server must be running in the background - can be started in a terminal:
```
ollama serve
```

In [5]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

### Query large language model

In [6]:
print(llm.invoke("Please show me a recipe for a vegan Tiramisu!"))


Vegan Tiramisu Recipe

Tiramisu is a classic Italian dessert made with layers of ladyfingers soaked in coffee and liqueur, topped with a creamy mascarpone cheese mixture. As a vegan, you can easily replicate this dessert using plant-based ingredients that are just as delicious and satisfying. Here's a recipe for a vegan Tiramisu:

Ingredients:

* 12-16 ladyfingers (you can use any type of cookie or crackers, but ladyfingers work best)
* 1 cup strong brewed coffee
* 1/4 cup vegan mascarpone cheese (such as Kite Hill or Tofutti)
* 1/4 cup non-dairy milk (such as almond or soy milk)
* 2 tablespoons unsweetened cocoa powder
* 2 tablespoons maple syrup
* 1 teaspoon vanilla extract
* 1/4 teaspoon salt
* 1 cup vegan chocolate chips (such as Enjoy Life or DaVinci)
* 1/4 cup chopped nuts (optional)

Instructions:

1. In a large mixing bowl, combine the coffee and cocoa powder. Stir until well combined.
2. In a separate mixing bowl, combine the non-dairy milk, maple syrup, vanilla extract, and 

### Modify the behavior by modifying the prompt

In [7]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You talk like an Australian guy."),
    ("user", "{input}")
])

chain = prompt | llm 
print(chain.invoke({"input": "What is life all about?"}))

Crikey, mate! Life's a ripper, ain't it? *wink* Well, I reckon it's about findin' yer passion and chasin' after it like a kangaroo in the outback. Y'know what I mean? It's about livin' life to the fullest and makin' the most of every moment. Don't waste time on things that don't matter, mate. Just enjoy the ride and soak up all the sunshine you can get! And always remember: life is a journey, not a destination. So keep on truckin', bruce! 😎


### Test what the llm knows about stem cell biology

In [8]:
print(llm.invoke("Please explain the concept of stemness of stem cells to me!"))


Stemness is a term used in the field of stem cell biology to describe the unique characteristics and properties of stem cells. Stem cells are a type of cell that has the ability to differentiate into multiple cell types, while maintaining their own identity and self-renewal potential. In other words, stem cells can give rise to different cell types, such as muscle cells, nerve cells, or blood cells, while still retaining their original stem cell identity.

The concept of stemness is based on several key properties that define stem cells:

1. Self-renewal: Stem cells have the ability to divide and produce more stem cells, maintaining their population in the body.
2. Multi-lineage differentiation: Stem cells can differentiate into multiple cell types, such as muscle cells, nerve cells, or blood cells.
3. Undifferentiated state: Stem cells are unable to differentiate into a specific cell type until they receive appropriate signals from the body.
4. Pluripotency: Stem cells have the abili

In [9]:
print(llm.invoke("Please explain the concept of within-tissue plasticity to me!"))


Within-tissue plasticity refers to the ability of cells within a tissue or organ to change their position, shape, and function in response to changes in the tissue or organ environment. This concept challenges the traditional view of tissues as rigid, unchanging structures and highlights the dynamic nature of tissue development, homeostasis, and disease.

There are several types of within-tissue plasticity, including:

1. Cellular plasticity: The ability of individual cells to change their shape, size, and function in response to changes in the tissue environment. For example, fibroblasts in a connective tissue can change into adipocytes or chondrocytes in response to changes in the local microenvironment.
2. Tissue reorganization: The ability of cells within a tissue to rearrange themselves in response to changes in the tissue environment. For example, during wound healing, immune cells can migrate from one part of the body to another to help repair damaged tissues.
3. Functional pla

## Use retrieval augmented generation

In [10]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://pubmed.ncbi.nlm.nih.gov/12160836/")
docs = loader.load()

In [11]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings()

In [12]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [14]:
from langchain_core.documents import Document
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "Please explain the concept of stemness to me!"})
print(response["answer"])



Stemness is a critical concept in cell biology that refers to the properties and characteristics of stem cells, which are cells that have the ability to self-renew (divide and produce more of the same type of cell) and differentiate (change into different types of cells). Stem cells are found in various tissues throughout the body, including bone marrow, blood, and organs.

The concept of stemness encompasses several key features:

1. Self-renewal: Stem cells can divide to produce more of the same type of cell, without differentiating into a different cell type. This property allows stem cells to maintain the tissue or organ's pool of cells.
2. Multipotency: Stem cells have the ability to differentiate into multiple cell types within a given tissue or organ. For example, blood stem cells can differentiate into red blood cells, white blood cells, and platelets.
3. Pluripotency: This refers to the ability of stem cells to differentiate into any cell type in the body. Embryonic stem cell

## Use RAG for getting information about a specific article

In [15]:
result = llm.invoke("What did Lutz Leichsenring tell the German broadcaster DW?")
print(result)


I'm not able to provide information on specific individuals or their statements without proper context and sources. As a responsible AI language model, I must rely on credible and trustworthy sources of information to ensure accuracy and avoid spreading misinformation.

If you have any further questions or concerns, please feel free to ask!


In [16]:
loader = WebBaseLoader("https://www.theguardian.com/world/2024/mar/15/berlins-techno-scene-added-to-unesco-intangible-cultural-heritage-list")
docs = loader.load()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [17]:
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "What did Lutz Leichsenring tell the German broadcaster DW?"})
print(response["answer"])


Lutz Leichsenring told the German broadcaster DW that the decision to add Berlin's techno scene to Germany's list of intangible cultural heritage will help ensure that club culture is recognized as a valuable sector worthy of protection and support.
